<a href="https://colab.research.google.com/github/zumaia/r/blob/master/webScraping_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial de web scraping en R


Ref.:  https://towardsdatascience.com/web-scraping-tutorial-in-r-5e71fd107f32

Hace un par de días, Kevin Markham de la Escuela de Datos, publicó un bonito tutorial sobre web scraping usando 16 líneas de código Python.

El web scraping de las mentiras del Presidente en 16 líneas de Python
Nota: Este tutorial está disponible como un cuaderno de Jupyter, y el conjunto de datos de las mentiras está disponible como un archivo CSV,...www.dataschool.io *texto en cursiva
http://www.dataschool.io/python-web-scraping-of-president-trumps-lies/
*

El tutorial es simple y muy bien hecho. Le recomiendo encarecidamente que le eche un vistazo. De hecho, tal tutorial me motivó a replicar los resultados pero esta vez usando R. Con el permiso de Kevin, usaré un diseño similar al de su entrada en el blog. También usaré el mismo sitio web sobre un artículo de opinión llamado Mentiras de Trump. Esto debería facilitar cualquier comparación entre los dos enfoques.

## Examinando el artículo del New York Times

Para una buena descripción del artículo con el que trabajaremos, te animo a que le eches un vistazo al tutorial de Kevin. En resumen, los datos que nos interesan consisten en un registro de mentiras, cada uno con 4 partes:

    La fecha de la mentira
    La mentira en sí misma
    Una explicación de por qué era una mentira
    Un URL para un artículo que apoye la explicación (incrustado en el texto)


## Leyendo la página web en R

Para leer la página web en R, podemos usar el paquete rvest, hecho por el gurú de R Hadley Wickham. Este paquete está inspirado en bibliotecas como Beautiful Soup, para facilitar el raspado de datos de páginas web html. La primera función importante a usar es read_html(), que devuelve un documento XML que contiene toda la información de la página web.

In [1]:
library(rvest)
webpage <- read_html("https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html")
webpage

Loading required package: xml2



{html_document}
<html lang="en" class="no-js page-interactive section-opinion page-theme-standard tone-opinion page-interactive-default limit-small layout-xlarge app-interactive" itemid="https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html" itemtype="http://schema.org/NewsArticle" itemscope="" xmlns:og="http://opengraphprotocol.org/schema/">
[1] <head>\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8 ...
[2] <body>\n<style>\n.lt-ie10 .messenger.suggestions {\n  display: block !imp ...

Recoger todos los registros

Como se explica en el tutorial de Kevin, cada registro tiene la siguiente estructura en el código HTML:

<span class="short-desc"><strong> DATE </strong> LIE <span class="short-truth"><a href="URL"> EXPLANATION </a></span></span>

Por lo tanto, para recoger todas las mentiras, necesitamos identificar todas las etiquetas <span> que pertenecen a class="shortdesc". La función que nos ayudará a hacerlo es html_nodes(). Esta función requiere el documento XML que hemos leído y los nodos que queremos seleccionar. Para esto último, se recomienda utilizar el SelectorGadget, una herramienta de código abierto que facilita la generación y el descubrimiento de los selectores de CSS. Usando dicha herramienta, encontramos que todas las mentiras pueden ser seleccionadas usando el selector ".short-desc".

In [2]:
results <- webpage %>% html_nodes(".short-desc")
results

{xml_nodeset (180)}
 [1] <span class="short-desc"><strong>Jan. 21 </strong>“I wasn't a fan of Ira ...
 [2] <span class="short-desc"><strong>Jan. 21 </strong>“A reporter for Time m ...
 [3] <span class="short-desc"><strong>Jan. 23 </strong>“Between 3 million and ...
 [4] <span class="short-desc"><strong>Jan. 25 </strong>“Now, the audience was ...
 [5] <span class="short-desc"><strong>Jan. 25 </strong>“Take a look at the Pe ...
 [6] <span class="short-desc"><strong>Jan. 25 </strong>“You had millions of p ...
 [7] <span class="short-desc"><strong>Jan. 25 </strong>“So, look, when Presid ...
 [8] <span class="short-desc"><strong>Jan. 26 </strong>“We've taken in tens o ...
 [9] <span class="short-desc"><strong>Jan. 26 </strong>“I cut off hundreds of ...
[10] <span class="short-desc"><strong>Jan. 28 </strong>“The coverage about me ...
[11] <span class="short-desc"><strong>Jan. 29 </strong>“The Cuban-Americans,  ...
[12] <span class="short-desc"><strong>Jan. 30 </strong>“Only 109 people out o 

Esto devuelve una lista con 116 nodos XML que contienen la información de cada una de las 116 mentiras de la página web.

Fíjate que estoy usando el %>% pipe-operator del paquete magrittr, que puede ayudar a expresar operaciones complejas como elegantes tuberías compuestas de piezas simples y fáciles de entender.

## Extrayendo la fecha

Comencemos con algo simple y concentrémonos en extraer todos los detalles necesarios de la primera mentira. Luego podemos extender esto a todos los demás fácilmente. Recuerden que la estructura general para un solo registro es:

<span class="short-desc"><strong> FECHA </strong> MENTIRA <span class="short-truth"><a href="URL"> EXPLICACIÓN </a></span></span>

Fíjese que la fecha está incrustada dentro de la etiqueta <strong>. Para seleccionarla, podemos usar la función html_nodes() usando el selector "strong".

In [3]:
first_result <- results[1]
first_result %>% html_nodes("strong")

{xml_nodeset (1)}
[1] <strong>Jan. 21 </strong>

Entonces necesitamos usar la función html_text() para extraer sólo el texto, con el argumento trim activo para recortar los espacios anteriores y posteriores. Finalmente, utilizamos el paquete stringr para añadir el año a la fecha extraída.

In [4]:
first_result <- results[1]
date <- first_result %>% html_nodes("strong") %>% html_text(trim = TRUE)

library(stringr)
str_c(date, ', 2017')

[1] "Jan. 21, 2017"

# Extrayendo la mentira

Para seleccionar la mentira, necesitamos hacer uso de la función xml_contents() que forma parte del paquete xml2 (este paquete es requerido por el paquete rvest, por lo que no es necesario cargarlo). La función devuelve una lista con los nodos que forman parte de first_result.

In [5]:
xml_contents(first_result)

{xml_nodeset (3)}
[1] <strong>Jan. 21 </strong>
[2] “I wasn't a fan of Iraq. I didn't want to go into Iraq.” 
[3] <span class="short-truth"><a href="https://www.buzzfeed.com/andrewkaczyns ...

Fíjese que hay un par de citas extra ("...") que rodean la mentira. Para deshacernos de ellas, simplemente usamos la función str_sub() del paquete stringr para seleccionar sólo la mentira.

In [6]:
lie <- xml_contents(first_result)[2] %>% html_text(trim = TRUE)
str_sub(lie, 2, -2)

[1] "I wasn't a fan of Iraq. I didn't want to go into Iraq."

# Extrayendo la explicación

Esperemos que a estas alturas no sea demasiado complicado ver que para extraer la explicación simplemente tenemos que seleccionar el texto dentro de la etiqueta <span> que pertenece a class=".short-truth". Esto extraerá el texto junto con los paréntesis de apertura y cierre, pero podemos deshacernos de ellos fácilmente.

In [7]:
explanation <- first_result %>% html_node(".short-truth") %>% html_text(trim = TRUE)
str_sub(explanation, 2, -2)

[1] "He was for an invasion before he was against it."

## Extrayendo el URL

Por último, para obtener la URL, noten que es un atributo dentro de la etiqueta <a>. Simplemente seleccionamos este nodo con la función html_nodes(), y luego seleccionamos el atributo href con la función html_attr().

In [8]:
url <- first_result %>% html_node("a") %>% html_attr("href")
url

[1] "https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the"

## Construir el conjunto de datos

Encontramos una forma de extraer cada una de las 4 partes del primer registro. Podemos extender este proceso a todos los demás usando un bucle de for. Al final, queremos tener un marco de datos con 116 filas (una para cada registro) y 4 columnas (para mantener la fecha, la mentira, la explicación y la URL). Una forma de hacerlo es crear un marco de datos vacío y simplemente añadir una nueva fila a medida que se procesa cada nuevo registro. Sin embargo, esto no se considera una buena práctica. Como se sugiere aquí, vamos a crear un solo marco de datos para cada registro y almacenarlos todos en una lista. Una vez que tengamos los 116 marcos de datos, los uniremos usando la función bind_rows() del paquete dplyr. Esto crea nuestro conjunto de datos deseado.

In [9]:
library(dplyr)
records <- vector("list", length = length(results))

for (i in seq_along(results)) {
    date <- str_c(results[i] %>% html_nodes("strong") %>% html_text(trim = TRUE), ", 2017")
    lie <- str_sub(xml_contents(results[i])[2] %>% html_text(trim = TRUE), 2, -2)
    explanation <- str_sub(results[i] %>% html_nodes(".short-truth") %>% html_text(trim = TRUE), 2, -2)
    url <- results[i] %>% html_nodes("a") %>% html_attr("href")
    records[[i]] <- data_frame(date = date, lie = lie, explanation = explanation, url = url)
}

df <- bind_rows(records)
glimpse(df)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“`data_frame()` is deprecated as of tibble 1.1.0.
Please use `tibble()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”


Rows: 180
Columns: 4
$ date        <chr> "Jan. 21, 2017", "Jan. 21, 2017", "Jan. 23, 2017", "Jan. …
$ lie         <chr> "I wasn't a fan of Iraq. I didn't want to go into Iraq.",…
$ explanation <chr> "He was for an invasion before he was against it.", "Trum…
$ url         <chr> "https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-…


Obsérvese que la columna de la fecha se considera un vector de carácter. Sería bueno tenerla como un vector de fecha y hora en su lugar. Para ello, podemos usar el paquete de lubridate y usar la función mdy() (mes-día-año) para hacer la conversión.

In [10]:
library(lubridate)
df$date <- mdy(df$date)
glimpse(df)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




Rows: 180
Columns: 4
$ date        <date> 2017-01-21, 2017-01-21, 2017-01-23, 2017-01-25, 2017-01-…
$ lie         <chr> "I wasn't a fan of Iraq. I didn't want to go into Iraq.",…
$ explanation <chr> "He was for an invasion before he was against it.", "Trum…
$ url         <chr> "https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-…


Exportar el conjunto de datos a un archivo CSV

Si quieres exportar tu conjunto de datos, puedes usar la función write.csv() que viene por defecto con R, o la función write_csv() del paquete readr, que es dos veces más rápida y conveniente que la primera.

In [12]:
readr::write_csv(df, "trump_lies.csv")

Del mismo modo, para recuperar su conjunto de datos, puede utilizar la función predeterminada read.csv() o la función read_csv() del paquete readr.

In [14]:
df <- readr::read_csv("trump_lies.csv")

Parsed with column specification:
cols(
  date = col_date(format = ""),
  lie = col_character(),
  explanation = col_character(),
  url = col_character()
)



In [15]:
head(df)

date,lie,explanation,url
<date>,<chr>,<chr>,<chr>
2017-01-21,I wasn't a fan of Iraq. I didn't want to go into Iraq.,He was for an invasion before he was against it.,https://www.buzzfeed.com/andrewkaczynski/in-2002-donald-trump-said-he-supported-invading-iraq-on-the
2017-01-21,A reporter for Time magazine — and I have been on their cover 14 or 15 times. I think we have the all-time record in the history of Time magazine.,Trump was on the cover 11 times and Nixon appeared 55 times.,http://nation.time.com/2013/11/06/10-things-you-didnt-know-about-time/
2017-01-23,Between 3 million and 5 million illegal votes caused me to lose the popular vote.,There's no evidence of illegal voting.,https://www.nytimes.com/2017/01/23/us/politics/donald-trump-congress-democrats.html
2017-01-25,"Now, the audience was the biggest ever. But this crowd was massive. Look how far back it goes. This crowd was massive.",Official aerial photos show Obama's 2009 inauguration was much more heavily attended.,https://www.nytimes.com/2017/01/21/us/politics/trump-white-house-briefing-inauguration-crowd-size.html
2017-01-25,Take a look at the Pew reports (which show voter fraud.),The report never mentioned voter fraud.,https://www.nytimes.com/2017/01/24/us/politics/unauthorized-immigrant-voting-trump-lie.html
2017-01-25,You had millions of people that now aren't insured anymore.,"The real number is less than 1 million, according to the Urban Institute.",https://www.nytimes.com/2017/03/13/us/politics/fact-check-trump-obamacare-health-care.html



Resumen

A continuación se muestra el código completo de este tutorial:

In [ ]:
# Load packages
library(rvest)
library(stringr)
library(dplyr)
library(lubridate)
library(readr)

# Read web page
webpage <- read_html("https://www.nytimes.com/interactive/2017/06/23/opinion/trumps-lies.html")

# Extract records info
results <- webpage %>% html_nodes(".short-desc")

# Building the dataset
records <- vector("list", length = length(results))

for (i in seq_along(results)) {
    date <- str_c(results[i] %>% 
                      html_nodes("strong") %>% 
                      html_text(trim = TRUE), ', 2017')
    lie <- str_sub(xml_contents(results[i])[2] %>% html_text(trim = TRUE), 2, -2)
    explanation <- str_sub(results[i] %>% 
                               html_nodes(".short-truth") %>% 
                               html_text(trim = TRUE), 2, -2)
    url <- results[i] %>% html_nodes("a") %>% html_attr("href")
    records[[i]] <- data_frame(date = date, lie = lie, explanation = explanation, url = url)
}

df <- bind_rows(records)

# Transform to datetime format
df$date <- mdy(df$date)

# Export to csv
write_csv(df, "trump_lies.csv")



También quiero mencionar que los paquetes de stringr, dplyr, lubricante y readr son todos parte de la familia tidyverse. Esta es una colección de paquetes R que están diseñados para trabajar juntos para hacer el proceso de análisis de datos más fácil. De hecho, también podrías usar el popular paquete purrr para evitar el bucle for. Sin embargo, esto requeriría la creación de una función que mapee cada registro a un marco de datos. Para otro ejemplo de cómo hacer web scraping, mira esta impresionante entrada de blog de Dean Attali.

Espero que encuentres este tutorial útil. Su propósito no es mostrar qué lenguaje de programación es mejor, sino aprender tanto de Python como de R, así como aumentar tus habilidades de programación y herramientas para abordar un conjunto más diverso de problemas.
